In [2]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import neuralnetwork as nn
from tfomics import utils, learn

# import models
from model_zoo import fourthplace_connectomics_model
from model_zoo import simple_connectomics_model, simple_connectomics_model2
from model_zoo import residual_connectomics_model, residual_connectomics_model2

# load data

In [3]:
filename = 'processed_dataset.hdf5'
#data_path = 'D:/Dropbox/TFconnect'
data_path = '/home/peter/Code/tensorflow/data'
filepath = os.path.join(data_path,filename)

group_name = ['processed_data']
dataset = h5py.File(filepath,'r')
%time dtf = np.array(dataset['/'+group_name[0]+'/dtf'])
ltf = np.array(dataset['/'+group_name[0]+'/ltf'])
dtf_crossval = np.array(dataset['/'+group_name[0]+'/dtf_crossval'])
ltf_crossval = np.array(dataset['/'+group_name[0]+'/ltf_crossval'])

X_train = dtf#[:10000,:,:,:]
y_train = ltf#[:10000,:]
X_valid = dtf_crossval#[:5000,:,:,:]
y_valid = ltf_crossval#[:5000,:]

CPU times: user 472 ms, sys: 1.2 s, total: 1.67 s
Wall time: 1.67 s


In [ ]:
X_train.shape

# fourth place competition

In [ ]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = fourthplace_connectomics_model.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, placeholders)
nnmodel.inspect_layers()

# set output file paths
results_path = utils.make_directory(data_path, 'results')
results_path = utils.make_directory(results_path, 'tfomics')
output_name = 'fourth_place'
filepath = os.path.join(results_path, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
X = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
X2 = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
data = {'train': X, 'valid': X2}
learn.train_minibatch(nntrainer, data, batch_size=1000, num_epochs=100, 
                    patience=15, verbose=2, shuffle=True)

In [ ]:
nntrainer.set_best_parameters()
validation_score(nntrainer, data_path)

In [ ]:
nntrainer.close_sess()

# Simple connectomics model

In [ ]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = simple_connectomics_model.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, placeholders)
nnmodel.inspect_layers()

# set output file paths
results_path = utils.make_directory(data_path, 'results')
results_path = utils.make_directory(results_path, 'tfomics')
output_name = '1d_version'
filepath = os.path.join(results_path, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
train = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=1000, num_epochs=200, 
                    patience=15, verbose=2, shuffle=True)

In [ ]:
nntrainer.set_best_parameters()
validation_score(nntrainer, data_path)

In [ ]:
nntrainer.close_sess()

# Simple connectomics model

In [ ]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = simple_connectomics_model2.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, placeholders)
nnmodel.inspect_layers()

# set output file paths
results_path = utils.make_directory(data_path, 'results')
results_path = utils.make_directory(results_path, 'tfomics')
output_name = '1d_version2'
filepath = os.path.join(results_path, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
X = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
X2 = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
data = {'train': X, 'valid': X2}
learn.train_minibatch(nntrainer, data, batch_size=100, num_epochs=100, 
                    patience=15, verbose=2, shuffle=True)

In [ ]:
learn.train_minibatch(nntrainer, data, batch_size=1000, num_epochs=100, 
                    patience=15, verbose=2, shuffle=True)

In [ ]:
nntrainer.set_best_parameters()
validation_score(nntrainer, data_path)

In [ ]:
nntrainer.close_sess()

# residual model

In [ ]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = residual_connectomics_model.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, placeholders)
nnmodel.inspect_layers()

# set output file paths
results_path = utils.make_directory(data_path, 'results')
results_path = utils.make_directory(results_path, 'tfomics')
output_name = 'residual'
filepath = os.path.join(results_path, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
train = {'inputs': X_train, 'targets': y_train, 'keep_prob': .8, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1., 'is_training': False}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=256, num_epochs=100, 
                    patience=10, verbose=2, shuffle=True)

In [ ]:
train = {'inputs': X_train, 'targets': y_train, 'keep_prob': .8, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1., 'is_training': False}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=256, num_epochs=100, 
                    patience=10, verbose=2, shuffle=True)

In [ ]:
nntrainer.set_best_parameters()
learn.train_minibatch(nntrainer, data, batch_size=2000, num_epochs=100, 
                    patience=10, verbose=2, shuffle=True)

In [ ]:
nntrainer.set_best_parameters()
validation_score(nntrainer, data_path)

In [ ]:
nntrainer.close_sess()

# residual model 2

In [4]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = residual_connectomics_model2.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, placeholders)
nnmodel.inspect_layers()

# set output file paths
results_path = utils.make_directory(data_path, 'results')
results_path = utils.make_directory(results_path, 'tfomics')
output_name = '1d_version_residual2'
filepath = os.path.join(results_path, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: input
(?, 3, 330, 1)
layer2: conv1
(?, 2, 326, 32)
layer3: conv1_batch
(?, 2, 326, 32)
layer4: conv1_active
(?, 2, 326, 32)
layer5: conv1_dropout
(?, 2, 326, 32)
layer6: resid1_1resid
(?, 2, 326, 32)
layer7: resid1_1resid_norm
(?, 2, 326, 32)
layer8: resid1_1resid_active
(?, 2, 326, 32)
layer9: resid1_2resid
(?, 2, 326, 32)
layer10: resid1_2resid_norm
(?, 2, 326, 32)
layer11: resid1_resid_sum
(?, 2, 326, 32)
layer12: resid1_resid
(?, 2, 326, 32)
layer13: resid1_batch
(?, 2, 326, 32)
layer14: resid1_dropout
(?, 2, 326, 32)
layer15: conv2
(?, 1, 322, 64)
layer16: conv2_batch
(?, 1, 322, 64)
layer17: conv2_active
(?, 1, 322, 64)
layer18: conv2_dropout
(?, 1, 322, 64)
layer19: resid2_1resid
(?, 1, 322, 64)
layer20: resid2_1resid_norm
(?, 1, 322, 64)
layer21: resid2_1resid_active
(?, 1, 322, 64)
layer22: resid

In [ ]:
train = {'inputs': X_train, 'targets': y_train, 'keep_prob_conv': 0.9, 'keep_prob_dense': 0.7, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob_conv': 1.0, 'keep_prob_dense': 1.0, 'is_training': False}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=100, num_epochs=200, 
                    patience=20, verbose=2, shuffle=True)


Epoch 1 out of 200 
[==============================] 100.0% -- time=0s -- loss=0.38352 -- accuracy=85.24%   
  valid loss:		0.35532
  valid accuracy:	0.86160+/-0.00000
  valid auc-roc:	0.92281+/-0.00000
  valid auc-pr:		0.91747+/-0.00398
  lower cross-validation found
  saving model to:  /home/peter/Code/tensorflow/data/results/tfomics/1d_version_residual2_best.ckpt
Epoch 2 out of 200 
[==============================] 100.0% -- time=0s -- loss=0.35810 -- accuracy=86.18%   
  valid loss:		0.34811
  valid accuracy:	0.86424+/-0.00000
  valid auc-roc:	0.92674+/-0.00000
  valid auc-pr:		0.92241+/-0.00243
  lower cross-validation found
  saving model to:  /home/peter/Code/tensorflow/data/results/tfomics/1d_version_residual2_best.ckpt
Epoch 3 out of 200 
[==============================] 100.0% -- time=0s -- loss=0.35341 -- accuracy=86.34%   
  valid loss:		0.34691
  valid accuracy:	0.86301+/-0.00000
  valid auc-roc:	0.92849+/-0.00000
  valid auc-pr:		0.92461+/-0.00218
  lower cross-validation

In [ ]:
train = {'inputs': X_train, 'targets': y_train, 'keep_prob_conv': 0.9, 'keep_prob_dense': 0.7, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob_conv': 1.0, 'keep_prob_dense': 1.0, 'is_training': False}
data = {'train': train, 'valid': valid}

nntrainer.set_best_parameters()
learn.train_minibatch(nntrainer, data, batch_size=500, num_epochs=50, 
                    patience=10, verbose=2, shuffle=True)

In [5]:
train = {'inputs': X_train, 'targets': y_train, 'keep_prob_conv': 0.9, 'keep_prob_dense': 0.7, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob_conv': 1.0, 'keep_prob_dense': 1.0, 'is_training': False}
data = {'train': train, 'valid': valid}

nntrainer.set_best_parameters()
learn.train_minibatch(nntrainer, data, batch_size=1000, num_epochs=50, 
                    patience=10, verbose=2, shuffle=True)

loading model from:  /home/peter/Code/tensorflow/data/results/tfomics/1d_version_residual2_best.ckpt
Epoch 1 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.33660 -- accuracy=86.82%   
  valid loss:		0.33640
  valid accuracy:	0.86764+/-0.00000
  valid auc-roc:	0.93344+/-0.00000
  valid auc-pr:		0.93055+/-0.00055
  lower cross-validation found
  saving model to:  /home/peter/Code/tensorflow/data/results/tfomics/1d_version_residual2_best.ckpt
Epoch 2 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.33278 -- accuracy=86.89%   
  valid loss:		0.33308
  valid accuracy:	0.86845+/-0.00000
  valid auc-roc:	0.93442+/-0.00000
  valid auc-pr:		0.93163+/-0.00008
  lower cross-validation found
  saving model to:  /home/peter/Code/tensorflow/data/results/tfomics/1d_version_residual2_best.ckpt
Epoch 3 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.33071 -- accuracy=86.95%   
  valid loss:		0.33845
  valid accuracy:	0.86843+/-0.00

KeyboardInterrupt: 

In [ ]:
nntrainer.set_best_parameters()
learn.train_minibatch(nntrainer, data, batch_size=1500, num_epochs=50, 
                    patience=10, verbose=2, shuffle=True)

In [4]:
train = {'inputs': X_train, 'targets': y_train, 'keep_prob_conv': 0.9, 'keep_prob_dense': 0.7, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob_conv': 1.0, 'keep_prob_dense': 1.0, 'is_training': False}
data = {'train': train, 'valid': valid}

nntrainer.set_best_parameters()
learn.train_minibatch(nntrainer, data, batch_size=2000, num_epochs=50, 
                    patience=10, verbose=2, shuffle=True)

loading model from:  /home/peter/Code/tensorflow/data/results/tfomics/1d_version_residual2_best.ckpt
Epoch 1 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.31479 -- accuracy=87.45%   
  valid loss:		0.34066
  valid accuracy:	0.86198+/-0.00000
  valid auc-roc:	0.94007+/-0.00000
  valid auc-pr:		0.93812+/-0.00319
  lower cross-validation found
  saving model to:  /home/peter/Code/tensorflow/data/results/tfomics/1d_version_residual2_best.ckpt
Epoch 2 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.31332 -- accuracy=87.50%   
  valid loss:		0.35571
  valid accuracy:	0.85311+/-0.00000
  valid auc-roc:	0.93967+/-0.00000
  valid auc-pr:		0.93782+/-0.00305
Epoch 3 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.31178 -- accuracy=87.57%   
  valid loss:		0.32350
  valid accuracy:	0.87277+/-0.00000
  valid auc-roc:	0.94090+/-0.00000
  valid auc-pr:		0.93886+/-0.00327
  lower cross-validation found
  saving model to:  /home/

KeyboardInterrupt: 

In [4]:
train = {'inputs': X_train, 'targets': y_train, 'keep_prob_conv': 0.9, 'keep_prob_dense': 0.9, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob_conv': 1.0, 'keep_prob_dense': 1.0, 'is_training': False}
data = {'train': train, 'valid': valid}

nntrainer.set_best_parameters()
learn.train_minibatch(nntrainer, data, batch_size=3000, num_epochs=50, 
                    patience=10, verbose=2, shuffle=True)

loading model from:  /home/peter/Code/tensorflow/data/results/tfomics/1d_version_residual2_best.ckpt
Epoch 1 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.30729 -- accuracy=87.75%   
  valid loss:		0.35023
  valid accuracy:	0.85236+/-0.00000
  valid auc-roc:	0.94121+/-0.00000
  valid auc-pr:		0.93929+/-0.00378
  lower cross-validation found
  saving model to:  /home/peter/Code/tensorflow/data/results/tfomics/1d_version_residual2_best.ckpt
Epoch 2 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.30537 -- accuracy=87.85%   
  valid loss:		0.32378
  valid accuracy:	0.86915+/-0.00000
  valid auc-roc:	0.94189+/-0.00000
  valid auc-pr:		0.93998+/-0.00396
  lower cross-validation found
  saving model to:  /home/peter/Code/tensorflow/data/results/tfomics/1d_version_residual2_best.ckpt
Epoch 3 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.30456 -- accuracy=87.85%   
  valid loss:		0.31722
  valid accuracy:	0.87389+/-0.00

KeyboardInterrupt: 

In [3]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = residual_connectomics_model2.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, placeholders)

# set output file paths
results_path = utils.make_directory(data_path, 'results')
results_path = utils.make_directory(results_path, 'tfomics')
output_name = '1d_version_residual2'
filepath = os.path.join(results_path, output_name)

# compile neural trainer
optimization['optimizer'] = 'momentum'
#optimization['use_nesterov'] = True
optimization['learning_rate'] = 0.0001
#optimization['momentum'] = 0.99

nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

train = {'inputs': X_train, 'targets': y_train, 'keep_prob_conv': .9, 'keep_prob_dense': 0.7, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob_conv': 1.0, 'keep_prob_dense': 1.0, 'is_training': False}
data = {'train': train, 'valid': valid}

nntrainer.set_best_parameters()
learn.train_minibatch(nntrainer, data, batch_size=3000, num_epochs=50, 
                    patience=10, verbose=2, shuffle=True)

loading model from:  /home/peter/Code/tensorflow/data/results/tfomics/1d_version_residual2_best.ckpt
Epoch 1 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.29338 -- accuracy=88.40%   
  valid loss:		0.31866
  valid accuracy:	0.87243+/-0.00000
  valid auc-roc:	0.94222+/-0.00000
  valid auc-pr:		0.94002+/-0.00497
  lower cross-validation found
  saving model to:  /home/peter/Code/tensorflow/data/results/tfomics/1d_version_residual2_best.ckpt
Epoch 2 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.29237 -- accuracy=88.46%   
  valid loss:		0.31971
  valid accuracy:	0.87208+/-0.00000
  valid auc-roc:	0.94218+/-0.00000
  valid auc-pr:		0.93997+/-0.00500
Epoch 3 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.29206 -- accuracy=88.45%   
  valid loss:		0.31886
  valid accuracy:	0.87247+/-0.00000
  valid auc-roc:	0.94232+/-0.00000
  valid auc-pr:		0.94013+/-0.00506
Epoch 4 out of 50 
[==============================] 100.0

KeyboardInterrupt: 

In [4]:


def validation_score(nntrainer, data_path):

    filename = 'valideval_dataset.hdf5'
    dataset = h5py.File(os.path.join(data_path,filename),'r')
    group_name = ['valid_data']
    val_dat = np.array(dataset['/'+group_name[0]+'/vs_valid'])
    val_lbl = np.array(dataset['/'+group_name[0]+'/label_valid'])

    fragLen = 330
    N = 14
    avg_F = np.mean(val_dat,axis=0)

    startgap = np.ceil(float(val_dat.shape[1] - fragLen)/N).astype('int')
    true_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')
    pred_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')

    # Counter for the "true_lbl" array
    cnt_ = 0
    # Counter for the "pred_lbl" array
    cnt_u = 0
    for a in range(val_dat.shape[0]):
        if a%100 == 0:
            print('\r' + 'X'*(a//100))

        # Create batch array to send thru network
        im_eval = np.empty((N*val_dat.shape[0],3,fragLen,1), dtype='float32')

        # Count the number of traces in each batch
        cnt = 0

        for b in range(val_dat.shape[0]):

            for n in range(0, val_dat.shape[1] - fragLen, startgap):
                try:
                    im_eval[cnt,:,:,0] = np.vstack((val_dat[a,n:n+fragLen],
                                         val_dat[b,n:n+fragLen],
                                         avg_F[n:n+fragLen]))
                except:
                    from IPython.core.debugger import Tracer
                    Tracer()()

                cnt += 1

            # Keep track of the true labels
            if val_lbl[a,b] == 1:
                true_lbl[cnt_] = 1
            else:
                true_lbl[cnt_] = 0

            cnt_ += 1

        # Run batch through network
        test = {'inputs': im_eval, 'keep_prob_conv': 1, 'keep_prob_dense': 1, 'is_training': False}
        pred_stop = nntrainer.get_activations(test, layer='output')[:,0]
        # Average output over each group of N traces
        for u in range(0, len(pred_stop), N):
            pred_lbl[cnt_u] = np.mean(pred_stop[u:u+N])
            cnt_u += 1        


    from sklearn.metrics import roc_curve, auc, precision_recall_curve, accuracy_score, roc_auc_score
    fpr, tpr, thresholds = roc_curve(true_lbl, pred_lbl)
    wrk = auc(fpr, tpr)
    print(wrk)
    return wrk

nntrainer.set_best_parameters()
validation_score(nntrainer, data_path)

loading model from:  /home/peter/Code/tensorflow/data/results/tfomics/1d_version_residual2_best.ckpt

X
XX
XXX
XXXX
XXXXX
XXXXXX
XXXXXXX
XXXXXXXX
XXXXXXXXX
0.937929762794


0.93792976279423879

In [6]:

def test_prediction(nntrainer, data_path):

    filename = 'competition_dataset_downsampled.hdf5'
    dataset = h5py.File(os.path.join(data_path,filename),'r')
    group_name = ['competition_data']
    val_dat = np.array(dataset['/'+group_name[0]+'/realval'])
    val_lbl = np.array(dataset['/'+group_name[0]+'/realtest'])

    fragLen = 330
    N = 14
    avg_F = np.mean(val_dat,axis=0)

    startgap = np.ceil(float(val_dat.shape[1] - fragLen)/N).astype('int')
    true_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')
    pred_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')

    # Counter for the "true_lbl" array
    cnt_ = 0
    # Counter for the "pred_lbl" array
    cnt_u = 0
    for a in range(val_dat.shape[0]):
        if a%100 == 0:
            print('\r' + 'X'*(a//100))

        # Create batch array to send thru network
        im_eval = np.empty((N*val_dat.shape[0],3,fragLen,1), dtype='float32')

        # Count the number of traces in each batch
        cnt = 0

        for b in range(val_dat.shape[0]):

            for n in range(0, val_dat.shape[1] - fragLen, startgap):
                try:
                    im_eval[cnt,:,:,0] = np.vstack((val_dat[a,n:n+fragLen],
                                         val_dat[b,n:n+fragLen],
                                         avg_F[n:n+fragLen]))
                except:
                    from IPython.core.debugger import Tracer
                    Tracer()()

                cnt += 1

            # Keep track of the true labels
            if val_lbl[a,b] == 1:
                true_lbl[cnt_] = 1
            else:
                true_lbl[cnt_] = 0

            cnt_ += 1

        # Run batch through network
        test = {'inputs': im_eval, 'keep_prob_conv': 1, 'keep_prob_dense': 1, 'is_training': False}
        pred_stop = nntrainer.get_activations(test, layer='output')[:,0]
        # Average output over each group of N traces
        for u in range(0, len(pred_stop), N):
            pred_lbl[cnt_u] = np.mean(pred_stop[u:u+N])
            cnt_u += 1        
    return pred_lbl


nntrainer.set_best_parameters()
pred_lbl = test_prediction(nntrainer, data_path)


loading model from:  /home/peter/Code/tensorflow/data/results/tfomics/1d_version_residual2_best.ckpt



ResourceExhaustedError: OOM when allocating tensor with shape[500,2,326,32]
	 [[Node: moments/sufficient_statistics/Square = Square[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](Conv2D)]]
	 [[Node: moments_11/sufficient_statistics/Shape/_347 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_406_moments_11/sufficient_statistics/Shape", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'moments/sufficient_statistics/Square', defined at:
  File "/home/peter/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/peter/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/peter/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  Fil